In [1]:
import os
import sys
sys.path.append('../')

from settings import config as cfg
from utils import io_utils as iou
from utils import waveform_plot_utils as wpu
from utils import fourier_math_utils as fmu
from settings import period_bounds as pb
from utils import general_display_utils as gdu

import pyaudio
import numpy as np
#import scipio
from scipy.io import wavfile
from scipy.io.wavfile import write
from io import BytesIO
from IPython.display import Audio, display, HTML
import matplotlib.pyplot as plt
from weasyprint import HTML
import sympy
import librosa
import pandas as pd
import re

from IPython.display import Audio
import ipywidgets as widgets

import plotly.graph_objs as go
from plotly.subplots import make_subplots
from IPython.display import display, clear_output
import math
import plotly.io as pio

Function that takes as a string mathematical function and extracts fourier coefficients and frequency

In [ ]:
def extract_a_b_f(function):
    a = b = frequency = 0
    try:
        # Extract the coefficient of the sine term
        match = re.search("(-?\d+\.\d+)[*]sin\((\d+\.\d+)[*]pi[*]t\)", function)
        if match:
            a = float(match.group(1))
            frequency = float(match.group(2))/2
        
        # Extract the coefficient of the cosine term
        match = re.search("(-?\d+\.\d+)[*]cos\((\d+\.\d+)[*]pi[*]t\)", function)
        if match:
            b = float(match.group(1))
            frequency = float(match.group(2))/2*1000
        
    except:
        pass
    return a, b, frequency

Function that produces the sound from the function in string format

In [ ]:
def produce_sound (functions, powers, name, limit_N):
    duration = 1.5
    total_power = np.sum(powers)
    samples_together = 0
    rate = 44100
    for i, func in enumerate(functions):
        t = sympy.symbols('t')
        function = sympy.sympify(func)
        a, b, f = extract_a_b_f(str(function))
        frequency = f
        amplitude = np.sqrt(a*a + b*b) * np.sqrt(powers[i+1] / total_power)
        t = np.linspace(0, duration, int(duration * rate))
        samples = amplitude * np.sin(2 * np.pi * frequency * t)
        if i >= 0:
            samples_together += samples
        if i < limit_N:
            audio = Audio(samples, rate=rate)
            display(audio)
            write(f"../results/analysed/f_harmonic_{i+1}_{name}.wav", rate, samples)

# Loading sounds

In [2]:
files = [os.path.join(cfg.PATH_INSTRUMENT_SAMPLES, name) for name in os.listdir(cfg.PATH_INSTRUMENT_SAMPLES)]
files.sort(key=lambda x: x.lower()) # making sure the order is the same as in period_bounds.py config file
sounds = []

for file in files:
    path = os.path.join(cfg.PATH_INSTRUMENT_SAMPLES, file)
    sound, rate = iou.load_sound(path)
    sounds.append((sound, rate))

/Users/duje/Documents/Lectures/2023_10_15_glazba_kroz_oci_znanstvenika/instruments_fourier_analysis/notebooks/../utils/io_utils.py:46: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, data = wavfile.read(filename)


# Playing sounds

In [3]:
iou.play_audio(files, n_columns=4)

# Plotting waveforms

In [4]:
wpu.plot_waveform(sounds, cfg.WAVEFORM_ZOOM_PERCENTAGES, files)

GridBox(children=(Checkbox(value=False, description='cello_c3'), Checkbox(value=False, description='clarinet_c…

# Extracting period

In [5]:
periods = []
rates = []

one_period_signals, sample_rates = fmu.extract_periods_and_data_rates(sounds)
one_period_audios = iou.export_and_store_one_period_audio(files, one_period_signals, sample_rates)

wpu.plot_waveform(sounds, cfg.WAVEFORM_ZOOM_PERCENTAGES, files, mark_one_period = True)

GridBox(children=(Checkbox(value=False, description='cello_c3'), Checkbox(value=False, description='clarinet_c…

# Calculating Fourier coefficients

In [6]:
fourier_coefficients_per_instrument = []

for one_period_signal, N in zip(one_period_signals, cfg.N_HARMONICS_PER_INSTRUMENT):
    fourier_coefficients_per_instrument.append(fmu.calculate_fourier_coefficients(one_period_signal, N))

# Representing signal as mathematical function

In [7]:
mathematical_representation_of_signal_per_instrument = []

for fourier_coefficients, period_bounds in zip(fourier_coefficients_per_instrument, pb.PERIOD_BOUNDS.values()):
    T = period_bounds[1] - period_bounds[0]
    mathematical_representation_of_signal_per_instrument.append(fmu.get_mathematical_representation_of_signal(fourier_coefficients, T))

gdu.print_mathematical_representation_of_signal(files, mathematical_representation_of_signal_per_instrument)

GridBox(children=(Checkbox(value=False, description='cello_c3'), Checkbox(value=False, description='clarinet_c…

# Reconstruct original signal

In [8]:
reconstructed_signals = []

for one_period_signal, fourier_coefficients in zip(one_period_signals, fourier_coefficients_per_instrument):
    reconstructed_signals.append(fmu.reconstruct_original_signal(one_period_signal, fourier_coefficients))
    
gdu.display_reconstructed_and_original_audio(files, reconstructed_signals, one_period_signals, sample_rates)

GridBox(children=(Checkbox(value=False, description='cello_c3'), Checkbox(value=False, description='clarinet_c…

# Drawing power spectra of harmonics

In [ ]:
relative_harmonic_powers_per_instrument = []

for fourier_coefficients in fourier_coefficients_per_instrument:
    relative_harmonic_powers_per_instrument.append(fmu.calculate_harmonic_power_spectrum(fourier_coefficients))
    
gdu.draw_harmonics_power_spectra(files, relative_harmonic_powers_per_instrument)

GridBox(children=(Checkbox(value=False, description='cello_c3'), Checkbox(value=True, description='clarinet_c5…

# Plotting individual harmonics

In [ ]:
gdu.plot_individual_harmonics(
    files, mathematical_representation_of_signal_per_instrument
)

GridBox(children=(Checkbox(value=False, description='cello_c3'), Checkbox(value=False, description='clarinet_c…

Preparing the plot. Please wait...


Preparing the plot. Please wait...


# Generate sound of each harmonic

In [ ]:
produce_sound(functions, powers, "sax_baritone", limit_N = 60)